# Xaar Waveform

## Configuration
-  Load waveform file into same folder as this notebook
-  Enter file name in variable `waveform_file`
-  Configuration dictionary should not change for all Xaar waveform files.

   `dict: {"data_block_name": [file_offset, block_length, invert]}`
  - `file_offset` - offset in bytes in the file (files is hex coded ascii)
  - `block_length` - length of the data block
  - `invert` - need the bytes in the 16bit words be swapped? (results in a nibble swap) 

## Usage
Example usage see below

In [1]:
# imports
import os

# Configuration
waveform_file = "./V405-100212-8LC-ITQ-CONFIDENTIAL.TXT"  # path to waveform file
waveform_size = 18412   # size of waveform file
row_file_offset = 4094  # offset in byte from row1 to row2 in file
out_path = './out/'     # file output destination foder
write_file = True       # write file block data to files

# dict value  [file offset, block length, invert]
waveform_elements_offset = {"Header": [0, 6, 0],
                            "CfgDB_row1": [6, 2040, 1],
                            "CfgDA_row1": [2046, 2040, 1],
                            "WaveformID_row1": [4086, 4, 0],
                            "SCLK_row1": [4090, 4, 0],
                            "CfgDB_row2": [0 + row_file_offset, 2040, 1],
                            "CfgDA_row2": [2040 + row_file_offset, 2040, 1],
                            "WaveformID_row2": [4080 + row_file_offset, 4, 0],
                            "SCLK_row2": [4084 + row_file_offset, 4, 0],
                            "Rest": [4084 + row_file_offset+4, 4, 0]}


In [2]:
def print_byte(pre, cnt, byte):
    """
    Print a byte in the different representations
    :param cnt:  element number
    :param byte: data byte
    :return:
    """
    # print("line: {}, val> chr:{} v:{} {}".format(str(cnt), byte, str(ord(byte)), hex(ord(byte))))
    print("| {0:12} | {1:10} | chr: {2:6} |".format(pre, str(cnt), str(byte)))


def write_line(data, filehandle):
    """
    Just another line writer, prints and write to file if given
    :param data:  string to print
    :param filehandle:  file handle
    :return:
    """
    print_data = ', '.join(data)
    print(print_data)
    if filehandle is not None and write_file:
        filehandle.write(print_data + '\n')

    
def read_n_print():
    """
    Read and print whole waveform file in a structured way.
    :return:
    """
    with open(waveform_file, 'rb') as waveform:
        # config
        line_len = 32

        # internal vars
        cnt = 0
        line_wrap = 0
        inv = 0
        data = []
        fh = None
        if not os.path.exists(out_path):
            os.makedirs(out_path)
        while cnt < waveform_size:
            byte_l = waveform.read(1)
            byte_h = waveform.read(1)
            if byte_l == '' or byte_h == '':
                print("file ends here")
                break

            for key, value in waveform_elements_offset.items():
                if value[0] == cnt:
                    write_line(data, fh)
                    data = []
                    line_wrap = 0
                    inv = value[2]
                    print("=== "+key+"==============")
                    if fh is not None:
                        fh.close()
                    fh = open(out_path + key + '.txt', 'w')

            if line_wrap >= line_len:
                write_line(data, fh)
                data = []
                line_wrap = 0

            if inv:
                data.append(byte_h.decode('UTF-8'))
                data.append(byte_l.decode('UTF-8'))
            else:
                data.append(byte_l.decode('UTF-8'))
                data.append(byte_h.decode('UTF-8'))
            line_wrap += 2

            # print("{0:2}{1:2}".format(byte_h.decode('UTF-8'), byte_l.decode('UTF-8')))
            cnt += 2

        # print rest if any
        write_line(data, fh)

        # close open files
        if fh is not None:
            fh.close()

        # end
        print('end: read_n_print')


# Program Code
# ===================================================
# File Read & Analyze
with open(waveform_file, 'rb') as waveform:

    print("| Parameter    | file line  | value       |")
    print("|--------------+------------+-------------|")
    # read waveform ID
    val_offset = 4086
    val_size = 4
    waveform.seek(val_offset, 0)
    b = waveform.read(val_size)
    print_byte("Waveform ID", val_offset, b)

    # read SCLK
    val_offset = 4090
    val_size = 4
    waveform.seek(val_offset, 0)
    b = waveform.read(val_size)
    print_byte("SCLK", val_offset, b)

# Print waveform data
read_n_print()


| Parameter    | file line  | value       |
|--------------+------------+-------------|
| Waveform ID  | 4086       | chr: b'V405' |
| SCLK         | 4090       | chr: b'0016' |

=== Header==============
0, 0, F, F, 8, 1
=== CfgDB_row1==============
0, 2, 1, 4, 6, 1, 8, 8, 8, 0, 8, 0, 8, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8
8, 8, 0, 8, 8, 0, 0, 8, 0, 8, 0, 8, 0, 0, 8, 0, 0, 0, 8, 8, 0, 0, 8, 8, 8, 0, 8, 0, 8, 0, 8, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 8, 8, 0, 0, 8, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0
8, 0, 0, 8, 8, 0, 8, 8, 8, 0, 8, 0, 8, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 0, 8, 8, 8, 0, 8, 8, 8, 8, 8, 0, 8, 0, 8, 0, 8, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 8
0, 8, 8, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0
0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0